# Introducing Mistral Large 2 (24.07)

[Mistral Large 2](https://mistral.ai/news/mistral-large-2407/) is an advanced large language model with state-of-the-art reasoning, knowledge, and coding capabilities according to Mistral. Mistral Large 2 is designed for single-node inference with long-context applications in mind – its size of 123 billion parameters allows it to run at large throughput on a single node. A significant effort was also devoted by Mistral to enhancing the model's reasoning capabilities. One of Mistral's key focuses during training was to minimize the model's tendency to "hallucinate," or generate plausible-sounding but factually incorrect or irrelevant information. This was achieved by fine-tuning the model to be more cautious and discerning in its responses, ensuring that it provides reliable and accurate outputs. Additionally, the new Mistral Large is trained to acknowledge when it cannot find solutions or does not have sufficient information to provide a confident answer. Mistral Large 2 sets a new frontier in terms of performance / cost of serving on evaluation metrics. In particular, on MMLU, the pretrained version achieves an accuracy of 84.0%, and sets a new point on the performance/cost Pareto front of open models.

The model is currently generally available for use on [Amazon Bedrock](https://aws.amazon.com/bedrock/), check out the [blog](https://aws.amazon.com/blogs/machine-learning/mistral-large-2-is-now-available-in-amazon-bedrock/) post for more info.

In this notebook we will be demonstrating a number of different features like multilingual capabilities (optimized for character-based languages), function calling and tool use, json output, math and reasoning, and code generation with Bedrock's [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html).

--- 
<div class="alert alert-block alert-info">
<b>IMPORTANT:</b>  Mistral Large 2 is being released under the <a href="https://mistral.ai/licenses/MRL-0.1.md"> Mistral Research License</a>, that allows usage and modification for research and non-commercial usages. If you need to make a commercial use of Mistral Large 2, you can purchase it as-a-Service on Bedrock or buy a license via SageMaker Jumpstart
</div>

---

<div class="alert alert-block alert-info">
<b>NOTE ON BENCHMARKS:</b> The below section goes over Benchmarks from Mistral published in their blog post for <a href="https://mistral.ai/news/mistral-large-2407/"> Mistral Large 2</a>. 
</div>

### Multi-lingual by design

Dozens of languages supported, including English, French, German, Spanish, Italian, Chinese, Japanese, Korean, Arabic, Portuguese, Dutch and Polish among others. In particular, it excels in English, French, German, Spanish, Italian, Portuguese, Dutch, Russian, Chinese, Japanese, Korean, Arabic, and Hindi. Below are the performance results of Mistral Large 2 on the multilingual MMLU benchmark from Mistral AI's official blog post on [Mistral Large 2](https://mistral.ai/news/mistral-large-2407/)

#### [Performance on Multilingual MMLU (measured on the base pretrained model) --benchmarked by Mistral AI](https://mistral.ai/images/news/mistral-large-2407/mistral-large-2407-language-diversity.png)

![multilingual-mmlu](imgs/mistral-large-2407-language-diversity.webp)


---
### Proficient in Code and Reasoning

Trained on 80+ coding languages such as Python, Java, C, C++, JavaScript, Bash, Swift, Fortran. Following their experience with Codestral 22B and Codestral Mamba, Mistral has trained Mistral Large 2 on a very large proportion of code. It vastly outperforms the previous Mistral Large. 

#### [Performance accuracy on code generation benchmarks (all models were benchmarked through the same evaluation pipeline) --benchmarked by Mistral AI](https://mistral.ai/images/news/mistral-large-2407/mistral-large-2407-code-generation.png)

![code-gen](imgs/mistral-large-2407-code-generation.png)


#### [Performance accuracy on MultiPL-E (all models were benchmarked through the same evaluation pipeline, except for the "paper" row) --benchmarked by Mistral AI](https://mistral.ai/images/news/mistral-large-2407/mistral-large-2407-multiple.png)

![code-multiple](imgs/mistral-large-2407-multiple.webp)

---
### [Agent-centric](https://mistral.ai/images/news/mistral-large-2407/mistral-large-2407-tool-use.png)

Best-in-class agentic capabilities with native function calling and JSON outputting. Mistral Large 2 is equipped with enhanced function calling and retrieval skills and has undergone training to proficiently execute both parallel and sequential function calls, enabling it to serve as the power engine of complex business applications.

---
## Model Card

<b>Available regions: US-West-2</b>

<b>Model ID: mistral.mistral-large-2407-v1:0</b>

<b>Context Window : 128k</b>

---

Let's get started with trying out the model on Bedrock!

#### Import necessary libraries

In [ ]:
import boto3
from boto3 import client
from botocore.config import Config
import json
import logging
from botocore.exceptions import ClientError
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

#### Create Bedrock Client

In [ ]:
config = Config(read_timeout=2000)
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2", config=config)

#### Instantiate model ids of mistral model being used

In [ ]:
mistral_large_2 = 'mistral.mistral-large-2407-v1:0'

model_id = mistral_large_2

### Generating JSON  & Tool Use

Mistral 2 now offers a native JSON output mode. This feature allows developers to receive the model's responses in a structured, easy-to-read format that can be readily integrated into various applications and systems. With JSON being a widely adopted data exchange standard, this capability simplifies the process of working with the model's outputs, making it more accessible and practical for developers across different domains and use cases.

To generate JSON with the Converse API you will need to define a toolSpec. Here we’ve outlined an example for a travel agent company that will take passenger information and requests and convert them to JSON.

Refer to this  article for more information on Generating JSON with Converse API : [Article](https://community.aws/content/2hWA16FSt2bIzKs0Z1fgJBwu589/generating-json-with-the-amazon-bedrock-converse-api?lang=en)

---

In [ ]:
# Define the tool configuration
import json
tool_list = [
    {
        "toolSpec": {
            "name": "travel_agent",
            "description": "Converts trip details as a json structure.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "origin_airport": {
                            "type": "string",
                            "description": "Origin airport (IATA code)"
                        },
                        "destination_airport": {
                            "type": "boolean",
                            "description": "Destination airport (IATA code)"
                        },
                        "departure_date": {
                            "type": "string",
                            "description": "Departure date",
                        }, 
                        "return_date": {
                            "type": "string",
                            "description": "Return date",
                        }
                    },
                    "required": [
                        "origin_airport",
                        "destination_airport",
                        "departure_date",
                        "return_date"
                    ]
                }
            }
        }
    }
]

In [ ]:
content = """
I would like to book a flight from New York (JFK) to London (LHR) for a round-trip.
The departure date is June 15, 2023, and the return date is June 25, 2023.

For the flight preferences, I would prefer to fly with Delta or United Airlines. 
My preferred departure time range is between 8 AM and 11 AM, and my preferred arrival time range is between 9 AM and 1 PM (local time in London). 
I am open to flights with one stop, but no more than that.
Please include non-stop flight options if available.
"""

message = {
    "role": "user",
    "content": [
        { "text": f"<content>{content}</content>" },
        { "text": "Please create a well-structured JSON object representing the flight booking request, ensuring proper nesting and organization of the data. Include sample data for better understanding. Create the JSON based on the content within the <content> tags." }
    ],
}

In [ ]:
# Bedrock client configuration
response = bedrock_client.converse(
    modelId=model_id,
    messages=[message],
    inferenceConfig={
        "maxTokens": 500,
        "temperature": 0.1
    },
    toolConfig={
        "tools": tool_list
    }
)

response_message = response['output']['message']
response_content_blocks = response_message['content']
content_block = next((block for block in response_content_blocks if 'toolUse' in block), None)
tool_use_block = content_block['toolUse']
tool_result_dict = tool_use_block['input']

print(json.dumps(tool_result_dict, indent=4))

Mistral Large 2 was able to correctly take our user query and convert the appropriate information to JSON. 


### Converse API & Tool Use

Mistral Large 2 also supports the Converse API & tool use. You can use the Amazon Bedrock API to give a model access to tools that can help it generate responses for messages that you send to the model. For example, you might have a chat application that lets users find out the most popular song played on a radio station. To answer a request for the most popular song, a model needs a tool that can query and return the song information. Here we have an example for getting the correct train schedule.

---

In [ ]:
# Define the tool configuration
toolConfig = {
    "tools": [
        {
            "toolSpec": {
                "name": "shinkansen_schedule",
                "description": "Fetches Shinkansen train schedule departure times for a specified station and time.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "station": {
                                "type": "string",
                                "description": "The station name."
                            },
                            "departure_time": {
                                "type": "string",
                                "description": "The departure time in HH:MM format."
                            }
                        },
                        "required": ["station", "departure_time"]
                    }
                }
            }
        }
    ]
}

In [ ]:
# Define the tool configuration
toolConfig = {
    "tools": [
        {
            "toolSpec": {
                "name": "shinkansen_schedule",
                "description": "Fetches Shinkansen train schedule departure times for a specified station and time.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "station": {
                                "type": "string",
                                "description": "The station name."
                            },
                            "departure_time": {
                                "type": "string",
                                "description": "The departure time in HH:MM format."
                            }
                        },
                        "required": ["station", "departure_time"]
                    }
                }
            }
        }
    ]
}

In [ ]:

def shinkansen_schedule(station, departure_time):
    schedule = {
        "Tokyo": {"09:00": "Hikari", "12:00": "Nozomi", "15:00": "Kodama"},
        "Osaka": {"10:00": "Nozomi", "13:00": "Hikari", "16:00": "Kodama"}
    }
    return schedule.get(station, {}).get(departure_time, "No train found")

In [ ]:
def prompt_mistral(prompt):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    converse_api_params = {
        "modelId": model_id,
        "messages": messages,
        "toolConfig": toolConfig,  
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    }

    response = bedrock_client.converse(**converse_api_params)
    
    if response['output']['message']['content'][0].get('toolUse'):
        tool_use = response['output']['message']['content'][0]['toolUse']
        tool_name = tool_use['name']
        tool_inputs = tool_use['input']

        if tool_name == "shinkansen_schedule":
            print("Mistral wants to use the shinkansen_schedule tool")
            station = tool_inputs["station"]
            departure_time = tool_inputs["departure_time"]
            
            try:
                result = shinkansen_schedule(station, departure_time)
                print("Train schedule result:", result)
            except ValueError as e:
                print(f"Error: {str(e)}")

    else:
        print("Mistral responded with:")
        print(response['output']['message']['content'][0]['text'])

In [ ]:
prompt_mistral("What train departs Tokyo at 9:00?")

Mistral Large 2 was able to correctly identify the shinkansen tool and demonstrate its use.

---
### Multilingual 

Mistral Large 2 now supports a large number of character-based languages such as Chinese, Japanese, Korean, Arabic, and Hindi. This expanded language support allows developers to build applications and services that can cater to users from diverse linguistic backgrounds. With multilingual capabilities, developers can create localized user interfaces, provide language-specific content and resources, and ensure a seamless experience for users regardless of their native language. In particular, Mistral Large 2 is efficienct with character-based languages.

Here, let's try to translate customer emails generated by the author into different languages such as Hindi, Arabic, Korean, & Japanese. 


In [ ]:
emails= """
"I recently bought your RGB gaming keyboard and absolutely love the customizable lighting features! Can you guide me on how to set up different profiles for each game I play?"
"I'm trying to use the macro keys on the gaming keyboard I just purchased, but they don't seem to be registering my inputs. Could you help me figure out what might be going wrong?"
"I'm considering buying your gaming keyboard and I'm curious about the key switch types. What options are available and what are their main differences?"
"I wanted to report a small issue where my keyboard's space bar is a bit squeaky. However, your quick-start guide was super helpful and I fixed it easily by following the lubrication tips. Just thought you might want to know!"
"My new gaming keyboard stopped working within a week of purchase. None of the keys respond, and the lights don't turn on. I need a solution or a replacement as soon as possible."
"I've noticed that the letters on the keys of my gaming keyboard are starting to fade after several months of use. Is this covered by the warranty?"
"I had an issue where my keyboard settings would reset every time I restarted my PC. I figured out it was due to a software conflict and resolved it by updating the firmware. Just wanted to ask if there are any new updates coming soon?"
"I've been having trouble with the keyboard software not saving my configurations, and it's starting to get frustrating. What can be done to ensure my settings are saved permanently?"
"""

In [ ]:
#prompts for language tasks/ translation

#japanese prompt
jpn_prompt=f"""

emails={emails}

Translate the following customer emails into Japanese. Your responses must be numbered, only in Japanese, and must adhere to only translating the emails.

""".format(emails=emails)

#korean prompt
korean_prompt=f"""

emails={emails}

Translate the following customer emails into Korean. Your responses must be numbered, only in Korean, and must adhere to only translating the emails.

""".format(emails=emails)

#hindi prompt
hindi_prompt=f"""

emails={emails}

Translate the following customer emails into Hindi. Your responses must be numbered, only in Hindi, and must adhere to only translating the emails.

""".format(emails=emails)

#arabic prompt
arabic_prompt=f"""

emails={emails}

Translate the following customer emails into Arabic. Your responses must be numbered, only in Arabic, and must adhere to only translating the emails.

""".format(emails=emails)

#### Mistral Large 2

---

In [ ]:
def converse(prompt, inference_config):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    response = bedrock_client.converse(
        messages=messages,
        modelId=model_id,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    print(generated_text)
    return generated_text, response

inference_config = {"temperature": 0.0, "maxTokens": 4000, "topP": 0.1}

#large 2 with hindi prompt
generated_text, response = converse(hindi_prompt, inference_config)

In [ ]:
#arabic prompt with large 2
generated_text, response = converse(arabic_prompt, inference_config)

In [ ]:
#japanese prompt with large 2
generated_text, response = converse(jpn_prompt, inference_config)

Based on the above responses, we can see that Mistral Large 2 performs really well with a number of character-based languages.

---

### Coding Tasks

Mistral Large 2 has been trained on over 80 coding languages, including popular ones like Python, Java, C, C++, JavaScript, and Bash, as well as more specialized languages such as Swift and Fortran. This comprehensive language support empowers developers to tackle a wide range of coding tasks and projects across various domains and platforms. Whether you're working on web development, mobile applications, scientific computing, or system programming, Mistral 2 can assist you with code generation, debugging, refactoring, and other coding-related tasks.

<mark><b>Sample coding tasks:</b></mark>

1. "Create a Python class for a multi-threaded web scraper that can handle rate limiting, proxy rotation, and dynamic content loading. Include methods for parsing HTML with BeautifulSoup and storing results in a SQLite database."
2. "Implement a Red-Black Tree data structure in C++ with methods for insertion, deletion, and rebalancing. Include a visualization function that prints the tree structure to the console."
3. "Write a Rust function that implements the Aho-Corasick string matching algorithm for efficient multi-pattern searching. Optimize it for memory usage and include comprehensive error handling."

In [ ]:
prompt = """
Write a Python function called palindrome_prime_finder that finds all prime numbers within a given range that are also palindromes when written in base 10 (decimal system).
"""

inference_config = {"temperature": 0.0, "maxTokens": 1000, "topP": 0.1}

response = converse(prompt, inference_config)

---
### Math and Reasoning Tasks

Mistral Large 2 excels at Math and Reasoning tasks, let's test some out:

<mark>    
<b>Sample Questions:</b></mark>
    
1. "Please provide a step-by-step reasoning process to estimate the number of stars in our galaxy, the Milky Way. Break down the calculation into logical steps, explaining your thought process and any assumptions you make along the way. Use scientific notation where appropriate, and conclude with a final estimate."
2. "What were the main reasons Frank Lloyd Wright designed his Oak Park Studio with high windows placed near the ceiling, and how did this feature reflect his architectural philosophy and the needs of an architecture firm's workspace?"
3. "Escape velocity from a neutron star: Given: A neutron star has a mass of 2.5 solar masses and a radius of 12 km. Task: Calculate the escape velocity from the surface of this neutron star in km/s."
    

In [ ]:
prompt = """
Calculating the orbital period of an exoplanet: Given: An exoplanet orbits its star at a distance of 2.5 AU (Astronomical Units). The star has a mass of 1.2 solar masses. Task: Calculate the orbital period of the exoplanet in Earth years.
"""

inference_config = {"temperature": 0.0, "maxTokens": 1000, "topP": 0.1}

response = converse(prompt, inference_config)

---
### Distributors

- Amazon Web Services
- Mistral AI